In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm
!python -m spacy download ru_core_news_sm
!python -m spacy download en_core_web_trf
!python -m spacy download ru_core_news_lg

!pip install beautifulsoup4
!pip install google
!pip install html2text

!pip install fuzzywuzzy
!pip install python-Levenshtein

     ---------------------------------------- 1.2/1.2 MB 1.3 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 62.3.3
    Uninstalling setuptools-62.3.3:
      Successfully uninstalled setuptools-62.3.3
     ---------------------------------------- 12.8/12.8 MB 5.5 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2022-06-13 15:22:12.991524: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-06-13 15:22:12.991560: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


     ---------------------------------------- 15.3/15.3 MB 5.8 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


2022-06-13 15:22:21.311270: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-06-13 15:22:21.311300: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# [+] Сделать запрос к поисковику, найти штук 10-20 ссылок

Собрать несколько текстов

https://www.geeksforgeeks.org/performing-google-search-using-python-code/

In [1]:
search_string = "Эльбрус-8С"
# search_string = "Байкал М"
# search_string = "Байкал М"

In [2]:
def search_google(search_string : str) -> (list, str):
    try:
        import googlesearch
        from googlesearch import search
    except ImportError:
        print("No module named 'google' found")
        return ([], "Не установлен python-модуль google")
    result = []
    try:
        result = googlesearch.search(search_string, tld="co.in", num=20, stop=20, pause=2)
    except:
        return ([], "Поиск в Google не удался, возможно, превышен лимит, попробуйте сменить IP или повторить позже")
    return (list(result), None)

search_result, err_message = search_google(search_string)
# for j in search_result:
#     print(j)
print(search_result)

['https://ru.wikipedia.org/wiki/%D0%AD%D0%BB%D1%8C%D0%B1%D1%80%D1%83%D1%81-8%D0%A1', 'https://ru.wikipedia.org/wiki/%D0%AD%D0%BB%D1%8C%D0%B1%D1%80%D1%83%D1%81_(%D0%BF%D1%80%D0%BE%D1%86%D0%B5%D1%81%D1%81%D0%BE%D1%80%D0%BD%D0%B0%D1%8F_%D0%B0%D1%80%D1%85%D0%B8%D1%82%D0%B5%D0%BA%D1%82%D1%83%D1%80%D0%B0)', 'https://ru.wikipedia.org/wiki/%D0%AD%D0%BB%D1%8C%D0%B1%D1%80%D1%83%D1%81-2%D0%A13', 'https://ru.wikipedia.org/wiki/%D0%AD%D0%BB%D1%8C%D0%B1%D1%80%D1%83%D1%81-4%D0%A1', 'https://ru.wikipedia.org/wiki/%D0%9C%D0%A6%D0%A1%D0%A2', 'http://www.mcst.ru/elbrus-8c', 'https://en.wikipedia.org/wiki/Elbrus-8S', 'http://www.mcst.ru/elbrus-8c0', 'http://www.mcst.ru/elbrus-8sv', 'https://habr.com/ru/post/647277/', 'https://www.ixbt.com/news/2021/05/30/s-t-a-l-k-e-r-8.html', 'https://www.tadviser.ru/index.php/%D0%9F%D1%80%D0%BE%D0%B4%D1%83%D0%BA%D1%82:%D0%AD%D0%BB%D1%8C%D0%B1%D1%80%D1%83%D1%81-8%D0%A1', 'https://3dnews.ru/1009369', 'https://servernews.ru/1057485', 'https://www.kommersant.ru/doc/5192750'

# [+] Получить страницы по ссылкам и выделить контексты

In [3]:
import re
import requests
# import bs4
# from bs4 import BeautifulSoup
# import webbrowser
import html2text
from tqdm.notebook import tqdm

def get_static_page_text(url : str) -> str:
    """
    Запрос к URL и выделение текста через requests.get().text
    """
    request_result=requests.get( url, timeout = 2.0)
    if request_result.status_code == 200:
        return str(request_result.text)
    else:
        return None

def clean_text(source_text : str):
    """
    Удаление переносов, табуляции, URL и прочих символов, мешающих
    распознаванию текста
    """
    cleaned_text = source_text.replace('\t', ' ')
    cleaned_text = cleaned_text.replace('\r', ' ')
    cleaned_text = cleaned_text.replace('\n', ' ')
    return cleaned_text

def get_static_page_text2(url : str) -> str:
    """
    Запрос к URL и выделение текста через html2text
    """
    request_result=requests.get( url )
    # raw=html2text.HTML2Text().handle(request_result.text)
    # raw=html2text.html2text(request_result.text)
    
    to_text = html2text.HTML2Text()
    to_text.ignore_links = True
    extracted_text = to_text.handle(request_result.text)
    
    # raw=html2text.html2text(request_result.content)
    # utf8_text=raw.encode('utf-8') # если нужен вывод в bytes
    return extracted_text # utf8_text если нужен вывод в bytes

def get_static_page_text3(url : str) -> str:
    """
    Запрос к URL и выделение текста через BeautifulSoup
    """
    pass

def get_word_contexts(
    source_text : str,
    target_text : str, # строка, для которой нужно выделить контекст
    words_before : int, # число слов перед найденным target_text, которые включаются в контекст
    words_after : int # число слов после найденного target_text, которые включаются в контекст
) -> list:
    """
    Выделение из текста source_text набора фрагментов текста (т.е. конеткстов в формате list(str)),
    включающих слово target_text, а также words_before слов перед словом target_text
    и words_after слов после слова target_text
    """
    # знаки препинания [\W&&\S]
    # слова с кириллицей [\wа-яА-Я]
    # regex_to_find_context = rf"(?:[\w]+\W+)\{{0,{words_before}}}{target_text}(?:\W+[\w]+)\{{0,{words_after}}}"
    regex_to_find_context = rf"(?:[\wа-яА-Я]+[^\wа-яА-Я]+){{0,{words_before}}}{target_text}(?:[^\wа-яА-Я]+[\wа-яА-Я]+){{0,{words_after}}}"
    # list_of_contexts = []
    # source_lower = source_text.lower()
    # target_lower = target_text.lower()
    # source_lower.find(target_lower)
    list_of_contexts = re.findall(regex_to_find_context, source_text, re.IGNORECASE)
    return list_of_contexts

# print(requests.get( search_result[0] ).text)

# text = ""
word_contexts = []
for url in tqdm(search_result, postfix="Страниц просканировано"):
    try:
        page_text=get_static_page_text2(url)
        tmp_word_contexts = get_word_contexts(source_text= page_text,
                                        target_text = search_string,
                                        words_before = 5,
                                        words_after = 5
                                       )
        word_contexts.extend(tmp_word_contexts)
        # soup = BeautifulSoup(html, features="html.parser")
        # text = text = ' '.join(soup.stripped_strings)
        # print("Исходный текст:\n\n", page_text,"\n\n")
        # print("Найденные совпадения:\n\n", word_contexts,"\n\n")
        # print("Число совпадений:\n\n", len(word_contexts),"\n\n")
    except:
        print("Для URL ", url, " не удалось получить страницу")
print(word_contexts)

  0%|          | 0/20 [00:00<?, ?it/s, Страниц просканировано]

Для URL  https://www.tadviser.ru/index.php/%D0%9F%D1%80%D0%BE%D0%B4%D1%83%D0%BA%D1%82:%D0%AD%D0%BB%D1%8C%D0%B1%D1%80%D1%83%D1%81-8%D0%A1  не удалось получить страницу
['Эльбрус-8С\n\nМатериал из Википедии — свободной энциклопедии', 'к навигации Перейти к поиску\n\nЭльбрус-8С  \n---  \n _Центральный процессор_  \n![Elbrus-8S.svg', 'Ядра |  \n  \nЭльбрус-4С\n\nЭльбрус-2С3  \n  \n**Эльбрус-8С** и **Эльбрус-8СВ** — 8-ядерные', 'МЦСТ[1].\n\nПервые опытные образцы Эльбрус-8С (1891ВМ10Я) были выпущены в 2014', 'разработка его сильно модернизированной версии —\nЭльбрус-8С', '4.1 Технические характеристики\n  * 5 Эльбрус-8С', 'частности,\nпроизводительность серверов на базе „Эльбрус-8С“ позволит в ближайшей\nперспективе приступить', 'Архитектура, схемотехника и топология микропроцессора «Эльбрус-8С» разработаны\nспециалистами Института электронных управляющих', '2014 идёт разработка новой модификации Эльбрус-8С под названием\n**Эльбрус-8С2** , в', 'и серверы) на базе процессора «Эльбрус-8С»[20]

# [+] Провести классификацию (найти категорию товара из списка категорий)

https://towardsdatascience.com/how-to-build-a-fast-most-similar-words-method-in-spacy-32ed104fe498

https://towardsdatascience.com/clustering-product-names-with-python-part-1-f9418f8705c8



In [10]:
import pandas as pd

# список категорий
df = pd.read_csv("База_продукты_Кирилл2.csv")
# table_names = ["Inventory", "HP BladeSystem Rack", "Network Interface"]
# groups = df[0].isin(table_names).cumsum()
# tables = {g.iloc[0,0]: g.iloc[1:] for k,g in df.groupby(groups)}

In [11]:
df.head()
#TODO классификация

,Unnamed: 0,name,category,subcategory,description,brand,params
0,0,Olesa Chugunova/Одежда/Рубашки/12333579,Женщинам,Блузки и рубашки,Качественная женская рубашка из тонкого хлопка...,Olesa Chugunova,;subj_name-Рубашки ;subj_root_name-Одежда
1,1,Рубашка Oversize/женская оверсайз/оверсайз белая,Женщинам,Блузки и рубашки,Женская рубашка. Рубашка из качественного мате...,Darya Selskaya,;subj_name-Рубашки ;subj_root_name-Одежда ;Вид...
2,2,Блузка женская одежда/Блуза/офисная,Женщинам,Блузки и рубашки,Блуза выполнена из мягкой струящейся ткани. Пр...,A-A Awesome Apparel by Ksenia Avakyan,;subj_name-Блузки ;subj_root_name-Одежда ;Длин...
3,3,Блузка,Женщинам,Блузки и рубашки,Блуза на каждый день с короткими свободными ру...,Pippinel,;subj_name-Блузки ;subj_root_name-Одежда ;Назн...
4,4,"Рубашка женская оверсайз вельвет, весенняя, ос...",Женщинам,Блузки и рубашки,"Рубашки вельвет это один из модных, стильных т...",Life in StyIe,;subj_name-Рубашки ;subj_root_name-Одежда ;Вид...


In [12]:
categories = pd.unique(df.iloc[:, 3]).tolist()

In [13]:
import spacy
from tqdm.notebook import tqdm

# nlp = spacy.load("en_core_web_sm")
# nlp = spacy.load("ru_core_news_sm")
nlp = spacy.load("ru_core_news_lg")

# text = "В частности, производительность серверов на базе „Эльбрус-8С“ позволит в ближайшей перспективе приступить к практическому построению на их основе суперкомпьютера"
# text = "Торговые названия – «микросхема 1891ВМ12Я» или «процессор “Эльбрус-8СВ”», также упоминается под названием Elbrus-8CV"
text = "тестирование двух типов серверов (двух- и четырёхпроцессорных), использовавших восьмиядерные микропроцессоры типа «Эльбрус-8С»"

categ_scores = [[categ_string, 0.0] for categ_string in categories]
for idx, categ_string in enumerate(tqdm(categories, postfix="Поиск наиболее близких категорий товаров")):
    category_doc = nlp(categ_string)
    source_doc = nlp(text)
    similarity_score = source_doc.similarity(category_doc)
    # categ_scores.append([categ_string, similarity_score])
    categ_scores[idx][1] += similarity_score
    # print("category = ", categ_string, ",\t score = ", similarity_score)
categ_scores.sort(key= lambda x:x[1], reverse=True)
print(categ_scores[:5])

  0%|          | 0/157 [00:00<?, ?it/s, Поиск наиболее близких категорий товаров]

C:\Users\1PC\AppData\Local\Temp\ipykernel_10440\2883823206.py:16: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_score = source_doc.similarity(category_doc)


[['Интернет и технологии', 0.4401719850519453], ['Запчасти для лодок и катеров', 0.4389801371958983], ['Ноутбуки и компьютеры', 0.43746715987211837], ['Кабели и зарядные устройства', 0.41760197451797515], ['Сборные конструкции', 0.37253160296842847]]


# [+] Выделить организации из результатов поиска и по частоте определить называние или страницу производителя

* named entity recognition (ner),
* прямой поиск из базы названий предприятий
* эластичный поиск

https://towardsdatascience.com/named-entity-recognition-with-nltk-and-spacy-8c4a7d88e7dahttps://towardsdatascience.com/named-entity-recognition-with-nltk-and-spacy-8c4a7d88e7da



## Выделение списка организаций

In [14]:
import spacy

# nlp = spacy.load("en_core_web_sm")
# nlp = spacy.load("ru_core_news_sm")
nlp = spacy.load("ru_core_news_lg")

# text = "Эльбрус-8С и Эльбрус-8СВ — 8-ядерные процессоры с архитектурой «Эльбрус» для персональных компьютеров и серверов. Разработаны российской компанией МЦСТ."
# text = "АО «НПП «Пульсар» входит в состав холдинга «Росэлектроника», управляющей компанией которого является Объединенная приборостроительная корпорация."
# text = "Apple is looking at buying U.K. startup for $1 billion"
# print("Исходный текст:\t", text,"\n")
doc = nlp(text)

# for token in doc:
#     # print(token)
#     # print(token.text, token.tag_)
#     # print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
#     #         token.shape_, token.is_alpha, token.is_stop)
orgs = []
stopwords = ["Википедия", "Википедии", "Коммерсантъ"]
for url in tqdm(search_result, postfix="Поиск названий организаций в результатах выдачи поисковика"):
    try:
        page_text=get_static_page_text2(url)
        
        doc = nlp(page_text)
        orgs_for_one_site = []
        # print("url = \t", url, "\n")
        for ent in doc.ents:
            if (ent.label_ == "ORG") and (ent.text not in stopwords):
                # print(ent.text, "\t", ent.label_)
                orgs_for_one_site.append(ent.text)
        # print("\n")
        orgs_for_one_site = list(dict.fromkeys(orgs_for_one_site))
        orgs.extend(orgs_for_one_site)
    except:
        continue
    
orgs.sort(key= lambda x:x[0])
print(orgs)
# https://spacy.io/usage/linguistic-features#named-entities
# print("Выделение категорий")
# for ent in doc.ents:
#     print(ent.text, "\t", ent.label_)

  0%|          | 0/20 [00:00<?, ?it/s, Поиск названий организаций в результатах выдачи поисковика]

['3ДНьюс', '3DNews', 'AMD', 'ALT Linux', 'Astra\nLinux Special Edition', 'ALC', 'AMD', 'ASML', 'AMD', 'ARK II', 'Apple', 'AD - Microsoft Active Directory Rights\nManagement Services - Microsoft Active Directory RMS', 'Acer', 'AV', 'BFM', 'BPM\n\n', 'C++', 'Cnews.ru', 'Creative Commons Attribution-ShareAlike', 'Creative Commons Attribution-ShareAlike', 'Creative Commons Attribution-ShareAlike', 'Creative Commons Attribution-ShareAlike', 'Creative Commons Attribution-ShareAlike', 'C++', 'CPU', 'Celsius', 'CPU', 'CNews', 'CNews.ru', 'CIO/Управление ИТ', 'Diablo', 'ECC', 'Esperanto Technologies[2', 'ECC', 'Elbrus Tech Day\n    ', 'Elbrus Tech Day\n    ', 'Elbrus Tech Day\n    ', 'Endless Dungeon', 'El Capitan', 'FPU', 'FCBGA', 'Fortran-77', 'Fortran', 'FCBGA', 'FCBGA', 'Forza Horizon', 'Ferrari', 'Facebook', 'Facebook VK Twitter Youtube Telegram\n\n![', 'Fiber-To-The-Building - MetroEthernet -\n', 'GTA', 'Golem.de', 'Global\nFoundries', 'Geely', 'GPU', 'Honda', 'HPC/Big Data', 'Hostkey', '

## Определение наиболее часто встречаемой

## Определение совпадений в базе

In [15]:
import pandas as pd

# список категорий
df2 = pd.read_csv("База_предприятия_Ксения2.csv", lineterminator="\n")

In [16]:
df2.head()
company_names_list = df2.iloc[:, 2].tolist()
# company_names_list

In [17]:
# https://towardsdatascience.com/fuzzywuzzy-find-similar-strings-within-one-column-in-a-pandas-data-frame-99f6c2a0c212

from fuzzywuzzy import process, fuzz
fuzzy_search_result = []
for organization_string in tqdm(orgs, postfix="Поиск совпадений организаций в базе по Москве"):
    fuzzy_search_result.extend(process.extract(organization_string, company_names_list, scorer=fuzz.token_sort_ratio))
fuzzy_search_result.sort(key= lambda x:x[1], reverse=True)
print(fuzzy_search_result[:10])

C:\Python39\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


  0%|          | 0/468 [00:00<?, ?it/s, Поиск совпадений организаций в базе по Москве]

[('КУБ', 80), ('КУБ', 80), ('Oil Distribution inc', 78), ('Oil Distribution inc', 78), ('Промтех', 77), ('Промтех', 77), ('Промтех', 77), ('Промтех', 77), ('Компания №1', 76), ('БФ КОМПАНИ', 76)]
